# **Setup Environment**

In [ ]:
# Change directory to the project root so that relative paths in .env 
# (e.g., SQLite db path) resolve correctly
%cd ../

In [ ]:
import sys
sys.path.append('.')

from app.db.async_db import MLModelService, MLMetricService
from app.db.models import DeviceType
from app.db.schemas import MLModelRequestCreate, MLModelRequestUpdate, MLMetricRequestCreate, MLMetricRequestUpdate

ml_model_service = MLModelService()
ml_metric_service = MLMetricService()

# **Test**

## **I. Test `ml_model_service`**

### **i. Test `create()` method**

In [ ]:
async def test_create_ml_model(name: str, device: DeviceType):
    try:
        model_data = MLModelRequestCreate(name=name, device=device)
        created_model = await ml_model_service.create(model_data)
        print(f"ML model successfully created:")
        print(created_model.model_dump_json(indent=2))
        
    except ValueError as e:
        print(f"Business logic error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

await test_create_ml_model(name="test_model_v1", device=DeviceType.CPU)

In [ ]:
# Trying to create an ML model with a name that already exists
await test_create_ml_model(name="test_model_v1", device=DeviceType.CUDA)

### **ii. Test `get()` method**

In [ ]:
async def test_get_ml_model_by_id(model_id: int):
    try:
        model = await ml_model_service.get(model_id)
        
        if model:
            print(f"ML model found:")
            print(model.model_dump_json(indent=2))
        else:
            print(f"ML model with ID {model_id} not found")
            
    except Exception as e:
        print(f"Error retrieving model: {e}")

await test_get_ml_model_by_id(1)

In [ ]:
# Trying to get an ML model with a non-existent ID
await test_get_ml_model_by_id(2)

### **iii. Test `get_all()` method**

In [ ]:
# Create a helper record to ensure get_all returns all entries, not just one
await test_create_ml_model(name="cool_model", device=DeviceType.CUDA)

In [ ]:
async def test_get_all_ml_models(skip: int = 0, limit: int = 10):
    try:
        models = await ml_model_service.get_all(skip=skip, limit=limit)
        print(f"Found {len(models)} ML models:")
        for model in models:
            print(model.model_dump_json(indent=2))
        
    except Exception as e:
        print(f"Error retrieving models: {e}")

await test_get_all_ml_models()

### **iv. Test `update()` method**

In [ ]:
async def test_update_ml_model(model_id: int, device: DeviceType):
    try:
        update_data = MLModelRequestUpdate(device=device)
        updated_model = await ml_model_service.update(model_id, update_data)

        if updated_model:
            print(f"ML model successfully updated:")
            print(updated_model.model_dump_json(indent=2))
        else:
            print(f"ML model with ID {model_id} not found")
    
    except ValueError as e:
        print(f"Business logic error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
        
await test_update_ml_model(model_id=1, device="cuda")

In [ ]:
await test_update_ml_model(model_id=1, device="cuda")

In [ ]:
# Trying to update an ML model with a non-existent ID
await test_update_ml_model(model_id=99, device="cuda")

### **v. Test `delete()` method**

In [ ]:
async def test_delete_ml_model(model_id: int):
    try:
        result = await ml_model_service.delete(model_id)
        
        if result:
            print(f"ML model with ID {model_id} successfully deleted")
        else:
            print(f"ML model with ID {model_id} not found")
        
    except Exception as e:
        print(f"Error deleting model: {e}")

await test_delete_ml_model(model_id=1)

In [ ]:
# # Trying to delete an ML model with a non-existent ID
await test_delete_ml_model(model_id=99)

## **II. Test `ml_metric_service`**

### **i. Test `create()` method**

In [ ]:
async def test_create_ml_metric(name: str, value: float, ml_model_id: int):
    try:
        metric_data = MLMetricRequestCreate(name=name, value=value, ml_model_id=ml_model_id)
        created_metric = await ml_metric_service.create(metric_data)
        print(f"ML metric successfully created:")
        print(created_metric.model_dump_json(indent=2))
        
    except ValueError as e:
        print(f"Business logic error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Assuming model with ID 2 exists (from previous test_create_ml_model)
await test_create_ml_metric(name="accuracy", value=0.95, ml_model_id=2)

In [ ]:
# Trying to create a metric for a non-existent model
await test_create_ml_metric(name="f1_score", value=0.88, ml_model_id=999)

In [ ]:
# Trying to create a duplicate metric for the same model
await test_create_ml_metric(name="accuracy", value=0.96, ml_model_id=2)

### **ii. Test `get()` method**

In [ ]:
async def test_get_ml_metric_by_id(metric_id: int):
    try:
        metric = await ml_metric_service.get(metric_id)
        
        if metric:
            print(f"ML metric found:")
            print(metric.model_dump_json(indent=2))
        else:
            print(f"ML metric with ID {metric_id} not found")
            
    except Exception as e:
        print(f"Error retrieving metric: {e}")

await test_get_ml_metric_by_id(1)

In [ ]:
# Trying to get a metric with a non-existent ID
await test_get_ml_metric_by_id(999)

### **iii. Test `get_all()` method**

In [ ]:
# Create another metric to ensure get_all returns multiple entries
await test_create_ml_metric(name="latency_ms", value=45.2, ml_model_id=2)

In [ ]:
async def test_get_all_ml_metrics(skip: int = 0, limit: int = 10):
    try:
        metrics = await ml_metric_service.get_all(skip=skip, limit=limit)
        print(f"Found {len(metrics)} ML metrics:")
        for metric in metrics:
            print(metric.model_dump_json(indent=2))
        
    except Exception as e:
        print(f"Error retrieving metrics: {e}")

await test_get_all_ml_metrics()

### **iv. Test `update()` method**

In [ ]:
async def test_update_ml_metric(metric_id: int, value: float):
    try:
        update_data = MLMetricRequestUpdate(value=value)
        updated_metric = await ml_metric_service.update(metric_id, update_data)

        if updated_metric:
            print(f"ML metric successfully updated:")
            print(updated_metric.model_dump_json(indent=2))
        else:
            print(f"ML metric with ID {metric_id} not found")
    
    except ValueError as e:
        print(f"Business logic error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
        
await test_update_ml_metric(metric_id=1, value=0.97)

In [ ]:
# Trying to update a metric with a non-existent ID
await test_update_ml_metric(metric_id=999, value=1.0)

### **v. Test `delete()` method**

In [ ]:
async def test_delete_ml_metric(metric_id: int):
    try:
        result = await ml_metric_service.delete(metric_id)
        
        if result:
            print(f"ML metric with ID {metric_id} successfully deleted")
        else:
            print(f"ML metric with ID {metric_id} not found")
        
    except Exception as e:
        print(f"Error deleting metric: {e}")

await test_delete_ml_metric(metric_id=1)

In [ ]:
# Trying to delete a metric with a non-existent ID
await test_delete_ml_metric(metric_id=999)